In [16]:
import pandas as pd
import numpy as np
import os
import matplotlib
import IPython
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d  
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes 
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
import time
from sympy import symbols, diff
from scipy.optimize import curve_fit
from eddington import fitting_function
from eddington import FittingData, fit
from eddington import linear
x, y, z = symbols('x y z', real=True)

In [17]:
path1A = r"C:\Users\forma\Programming\Jupyter Projects\שיעורי בית\מעבדה\מגנטיות\radios"
path1B = r"C:\Users\forma\Programming\Jupyter Projects\שיעורי בית\מעבדה\מגנטיות\angle"
path2 = r"C:\Users\forma\Programming\Jupyter Projects\שיעורי בית\מעבדה\מגנטיות\partb"
os.chdir(path2)

In [18]:
mu0=4*np.pi*10**(-7)

In [19]:
#range=30-85
def impdata1(n):
    os.chdir(path1A)
    TIME=[]
    MAG=[]
    count=0
    try:
        with open('{}cm.csv'.format(n)) as f:
            for line in f.readlines():
                line=line.split(',')
                #print(line)
                if count == 0:
                    count=+1
                    continue
                else: 
                    None
                    TIME.append(float(line[0]))
                    MAG.append(float(line[1].strip('\n'))*10**-3)
            return (np.array(TIME), np.array(MAG),n)
    except:
        None

In [20]:
#R=constant=45cm
#range=0-360
def impdata1B(n):
    os.chdir(path1B)
    TIME=[]
    MAG=[]
    count=0
    try:
        with open('{}d.csv'.format(n)) as f:
            for line in f.readlines():
                line=line.split(',')
                #print(line)
                if count == 0:
                    count=+1
                    continue
                else: 
                    None
                    TIME.append(float(line[0]))
                    MAG.append(float(line[1].strip('\n'))*10**-3)
            return (np.array(TIME), np.array(MAG),n)
    except:
        None

In [21]:
#range1-10
def impdata2B(n):
    os.chdir(path2)
    TIME=[]
    MAG=[]
    count=0
    try:
        with open('{}b.csv'.format(n)) as f:
            for line in f.readlines():
                line=line.split(',')
                #print(line)
                if count == 0:
                    count=+1
                    continue
                else: 
                    None
                    TIME.append(float(line[0]))
                    MAG.append(float(line[1].strip('\n')))
            return (np.array(TIME), np.array(MAG),n)
    except:
        None

In [22]:
def Erdata():
    os.chdir(path1A)
    TIME=[]
    MAG=[]
    count=0
    try:
        with open('Earth.csv') as f:
            for line in f.readlines():
                line=line.split(',')
                #print(line)
                if count == 0:
                    count=+1
                    continue
                else: 
                    None
                    TIME.append(float(line[0]))
                    MAG.append(float(line[1].strip('\n'))*10**-3)
            return (np.array(TIME), np.array(MAG))
    except:
        None

In [23]:
data1a=[]
for i in range(1,90):
    if impdata1(i)!=None:
        TIME,MAG,R = impdata1(i)
        MEAN=np.mean(MAG)
        STD=np.std(MAG)
        data1a.append((MEAN,STD,R))

In [24]:
data2a=[]
for i in range(0,360):
    if impdata1B(i) != None:
        TIME,MAG,d = impdata1B(i)
        MEAN=np.mean(MAG)
        STD=np.std(MAG)
        data2a.append((MEAN,STD,i))

In [25]:
TIME,ERB=Erdata()
MEANEARTH=np.mean(ERB)
STDEARTH=np.std(ERB)
err=statres(STDEARTH,50,1*10**-6)
print("Bearth: ",MEANEARTH,'\n','Error Bearth: ',err)


Bearth:  4.636e-05 
 Error Bearth:  2.965490403513951e-07


In [26]:
def RES(resu):
    return resu/(12)**0.5    

In [27]:
def stat(std,N):
    return std/(N)**0.5

In [28]:
def statres(std,N,res):
    st=stat(std,N)
    rs=RES(res)
    err = (st**2 + rs**2)**0.5
    return err

In [29]:
def AKIFA(func, x, errx, y, yerr, z, zerr):
    if x!=None and errx!=None:
        dx=(diff(func, x)*errx)**2
    else:
        dx=0
    if y!=None and yerr!=None:
        dy=(diff( func, y)*erry)**2
    else:
        dy=0
    if z!=None and zerr!=None:
        dz=(diff( func, z)*errz)**2
    else:
        dz=0
    return((dx+dy+dz)**0.5)

In [30]:
def get_Akifa_array(func=None,x_data=None, errx=None,y_data=None, yerr=None,z_data=None, zerr=None):                
    err_data=[]
    akifa=AKIFA(func,x,errx,y,yerr,z,zerr)
    
    if x_data==None and y_data==None and z_data==None:
        print("No vars given")
        
    if x_data!=None and y_data==None and z_data==None:
        for i in x_data:
            err_data.append(akifa.subs(x,i))
            
    if x_data!=None and y_data!=None and z_data==None:
        for i in range(x_data):
            err_data.append(akifa.subs({x:x_data[i],y:y_data[i]}))
            
    if x_data!=None and y_data!=None and z_data!=None:
        for i in range(datax_array):
            err_data.append(akifa.subs({x:x_data[i],y:y_data[i],z:z_data[i]}))
    print(err_data)

In [31]:
y=[]
y_error=[]
x=[]
x_error=[]
for i in data1a:
    y.append((i[0]-MEANEARTH)/mu0)
    y_error.append((abs(statres(i[1],50,1*10**-6)**2+statres(STDEARTH,50,1*10**-6)**2)**0.5)/mu0)
    x.append(i[2])
    x_error.append(RES(0.001))
df1A = pd.DataFrame ({"x": np.array(x)/100,"x_error":x_error ,"y": y,"y_error":y_error})

In [32]:
y=[]
y_error=[]
x=[]
x_error=[]
for i in data2a:
    y.append((i[0]-MEANEARTH)/mu0)
    y_error.append((abs((statres(i[1],50,1*10**-6)**2+statres(STDEARTH,50,1*10**-6)**2))**0.5)/mu0)
    x.append(i[2]*np.pi/180)
    x_error.append(RES(np.pi/180*np.sqrt(12)))
df1B = pd.DataFrame ({"x": x,"x_error":x_error ,"y": y,"y_error":y_error})

In [33]:
def findstats(array):
    picks_ind=[]
    picks=[]
    for i in range(2,len(array)-2):
        if np.all(array[i]>array[i-2:i]) and np.all(array[i]>array[i+1:i+2]):
            picks_ind.append(i)
        if np.all(array[i]<array[i-2:i]) and np.all(array[i]<array[i+1:i+2]):
            picks_ind.append(i)
    return picks_ind

In [34]:
def findind(arr):
    picks_ind1=[]
    picks=[]
    move=5
    for i in range(move,len(arr)-move):
        
        if np.all(arr[i]<arr[i-move:i]) and np.all(arr[i]<arr[i+1:i+move]):

            picks_ind1.append(i)

    return picks_ind1

In [35]:
lst=[]
def calc_from_index(indexes,data_array):
    for i in range(len(indexes)-1):
        point = abs(data_array[indexes[i+1]]-data_array[indexes[i]])
        lst.append(np.array(point))
    return(lst)

In [36]:
def AVG(lst):
    new_lst=[]
    for i in range(5,len(lst)-6):
        new_lst.append(sum(lst[i-5:i+5])/10)
    return new_lst
MAGAVG=AVG(MAG[50:-50])

In [37]:
def plot_chack(num_pics,MAGAVG,TIME):
    lst_of_AMPS=[]
    lst_of_TEMPS_inMAX_AMPS=[]
    lst_of_TEMPS_inMIN_AMPS=[]
    for n in range(0,num_pics):
        #fig, ax1 = plt.subplots(figsize=(5,3))
        #color = 'tab:blue'
        #ax1.set(title='{}'.format(n))
        #ax1.set_xlabel(r'$TIME$')
        #ax1.set_ylabel('MAGDIELD', color=color)
        #ax1.plot(TIME[n*len(MAGAVG)//num_pics:(n+1)*len(MAGAVG)//num_pics],
        #         MAGAVG[n*len(MAGAVG)//num_pics:(n+1)*len(MAGAVG)//num_pics], color=color)
        #print("=============================================================================================")
        lst_of_AMPS.append(np.max(MAGAVG[n*len(MAGAVG)//num_pics:(n+1)*len(MAGAVG)//
                                         num_pics])-np.min(MAGAVG[n*len(MAGAVG)//num_pics:(n+1)*len(MAGAVG)//num_pics]))
        lst_of_TEMPS_inMAX_AMPS.append(np.argmax(MAGAVG[n*len(MAGAVG)//
                                                        num_pics:(n+1)*len(MAGAVG)//num_pics])+n*len(MAGAVG)//num_pics)
        lst_of_TEMPS_inMIN_AMPS.append(np.argmin(MAGAVG[n*len(MAGAVG)//
                                                        num_pics:(n+1)*len(MAGAVG)//num_pics])+n*len(MAGAVG)//num_pics)
        
        temp_of_max=[]
        for i in lst_of_TEMPS_inMAX_AMPS:
            temp_of_max.append(TIME[i])
              
        temp_of_min=[]
        for i in lst_of_TEMPS_inMIN_AMPS:
            temp_of_min.append(TIME[i])
    time_in_max=np.array(temp_of_max)   
    time_in_min=np.array(temp_of_min)
    AMPS=np.array(lst_of_AMPS)
    AMPS=AMPS/2
    NA=len(AMPS)
    #print('you need to check if this points are coesponding to the graph\'s:',"\n",
    #     "the times in max points are:",time_in_max,'\n',
    #     "the times in min points:",time_in_min,'\n',
    #     'the Ampl are:',AMPS,'\n',
    #     "Number N of AMPL sumpples are:",N)
    return(time_in_max,time_in_min,AMPS,NA)

In [38]:
results=[('T','STDT','AMP','STDAMP','NA','NT')]
for i in range(1,11):    
    pics_list=[6,8,11,14,19,25,31,39,51,58]
    TIME,MAG,n=impdata2B(i)
    MAGAVG=AVG(MAG[120:-110])
    time_in_max,time_in_min,AMPS,NA=plot_chack(pics_list[i-1],MAGAVG,TIME)
    
    TFRAX=np.ediff1d(time_in_max)
    TFRIN=np.ediff1d(time_in_min)
    Ttotal=np.concatenate([TFRAX, TFRIN])
    NT=len(Ttotal)
    T=np.mean(Ttotal)
    STDT=np.std(Ttotal)
    AMP=np.mean(AMPS)
    STDAMP=np.std(AMPS)
    results.append((T,STDT,AMP,STDAMP,NA,NT))
    
    #fig, ax1 = plt.subplots(figsize=(15,3))
    #color = 'tab:blue'
    #ax1.set(title='{}'.format(n))
    #ax1.set_xlabel(r'$TIME$')
    #ax1.set_ylabel('MAGDIELD', color=color)
    #ax1.plot(TIME,MAG, color=color)
df = pd.DataFrame(results)

In [39]:
T=np.array(df[0])[1:]
TSTD=np.array(df[1])[1:]
AMP=np.array(df[2])[1:]
AMPSTD=np.array(df[3])[1:]
NA=np.array(df[4])[1:]
NT=np.array(df[5])[1:]



stat_T = TSTD/(NT)**0.5
stat_A = AMPSTD/(NA)**0.5

res_t=RES(1/500)
res_v=RES(0.001)
resu_tt=np.sqrt(2)*res_t
resu_vt=np.sqrt(2)*res_v/2
T_err=(stat_T**2+resu_tt**2)**0.5


omega = (2*np.pi)/T

x=omega
y=AMP

y_error = (stat_A**2+resu_vt**2)**0.5
x_error = (2*np.pi*T_err)/T**2

In [40]:
df2 = pd.DataFrame ({"x": x,"x_error":x_error ,"y": y,"y_error":y_error})

In [41]:
print("THE final data to process in"+ 
      "the presnttion of 10 first lines:")

print('\n',"the first part first part data with errors are:",'\n')

print(df1A[:10])

print('\n',"the first part second part data with errors are:",'\n')

print(df1B[:10])

print('\n',"the second  data with errors are:",'\n')

print(df2)

THE final data to process inthe presnttion of 10 first lines:

 the first part first part data with errors are: 

      x   x_error          y   y_error
0  0.30  0.000289  51.629864  0.333583
1  0.31  0.000289  43.321976  0.333158
2  0.32  0.000289  36.462397  0.332160
3  0.33  0.000289  30.637327  0.331641
4  0.34  0.000289  26.547045  0.333188
5  0.35  0.000289  22.631833  0.332396
6  0.36  0.000289  19.607889  0.329334
7  0.37  0.000289  16.695354  0.333621
8  0.38  0.000289  14.562677  0.333621
9  0.39  0.000289  13.241691  0.329334

 the first part second part data with errors are: 

          x   x_error         y   y_error
0  0.000000  0.017453  6.764085  0.331641
1  0.261799  0.017453  6.700423  0.332617
2  0.523599  0.017453  5.474930  0.332823
3  0.785398  0.017453  4.631409  0.332922
4  1.047198  0.017453  2.594226  0.333834
5  1.308997  0.017453  0.954930  0.334038
6  1.570796  0.017453 -0.302394  0.329711
7  1.832596  0.017453 -2.435071  0.333348
8  2.094395  0.017453 -4.0

In [42]:
path21A=r'C:\Users\forma\Programming\Jupyter Projects\שיעורי בית\מעבדה\מגנטיות\PART11.csv'
path21B=r'C:\Users\forma\Programming\Jupyter Projects\שיעורי בית\מעבדה\מגנטיות\PART12.csv'
path22=r'C:\Users\forma\Programming\Jupyter Projects\שיעורי בית\מעבדה\מגנטיות\PART2.csv'

df1A.to_csv(path21A, index = False)
df1B.to_csv(path21B, index = False)
df2.to_csv(path22, index = False)

In [43]:

@fitting_function(n=3,save=False)
def PART111(a, x):
    return ((a[0]) / (x + a[1])**3) + a[2]
@fitting_function(n=3,save=False)
def PART12(a, x):
    return a[0]*np.cos(x + a[1]) + a[2]